<a href="https://colab.research.google.com/github/gabrielebilliciani/knowledge-distillation/blob/main/Notebook3_KD_Classification_CNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Knowledge Distillation for Classification

 > **Note**: This notebook is part of an introductory project on Knowledge Distillation. It serves as a practical companion to the main presentation slides. The complete project is available on GitHub at: https://github.com/gabrielebilliciani/knowledge-distillation

This notebook acts as a follow-up to the previous one (available at https://github.com/gabrielebilliciani/knowledge-distillation/blob/main/Notebook2_KD_Classification_MLPs.ipynb), where we have introduced an application of Knowledge Distillation (KD) to classification tasks using Multi-Layer Perceptrons (MLPs) on MNIST data set.

This notebook is structured as a series of experiments to further explore the potential of KD in classification tasks. We had concluded that, despite finding statistically significant improvements in the performance of the distilled student model, the MNIST data set might have been too simple to fully demonstrate the power of KD, since the baseline student model already achieved very high accuracy on its own.

In order to explore more complex classification tasks, we will here use the CIFAR-10 and CIFAR-100 data sets, which are more challenging and require more sophisticated models. We will also use Convolutional Neural Networks (CNNs) as both teacher and student models, as they are better suited for image classification tasks.
1.  **CIFAR-10 Basics:** First, we'll distil a pre-trained ResNet-56 into a simple CNN on the CIFAR-10 dataset.
2.  **CIFAR-100 Challenge:** We'll move to the more difficult CIFAR-100 dataset and use a more capable ResNet-18 as student to see how a better architecture can better absorb the teacher's knowledge.

*Some aspects of the code are similar to the previous notebook, so we will not repeat their explanations here.*

# CIFAR-10

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from tqdm.notebook import tqdm
import numpy as np
import copy
import matplotlib.pyplot as plt

# --- Configuration ---
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
EPOCHS = 20
PATIENCE = 5
BATCH_SIZE = 64
LR = 0.001
TEMPERATURE = 10
ALPHA = 0.2

print(f"Using device: {DEVICE}")

Using device: cuda


In [ ]:
# --- Data Loading and Preprocessing ---
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

full_train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Split training data into train and validation sets
train_size = int(0.9 * len(full_train_dataset))
val_size = len(full_train_dataset) - train_size
train_subset, val_subset = random_split(full_train_dataset, [train_size, val_size])

# Create DataLoaders
train_loader = DataLoader(train_subset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"Train samples: {len(train_subset)}, Validation samples: {len(val_subset)}")

100%|██████████| 170M/170M [00:03<00:00, 50.0MB/s]


Train samples: 45000, Validation samples: 5000


## Model Architecture

Here we define the simple Convolutional Neural Network (CNN) that will act as our student model.

In [ ]:
class StudentNet(nn.Module):
    def __init__(self):
        super(StudentNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32 * 8 * 8, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 8 * 8)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

## Training and Evaluation Functions

These functions handle the core logic for the distillation loss, training loops, and model evaluation.

To prevent overfitting and ensure we select the best possible version of each model, we employ an **early stopping** mechanism based on the model's performance on a held-out **validation set**.

In [ ]:
def distillation_loss(student_outputs, labels, teacher_outputs, T, alpha):
    """
    Calculates the distillation loss, combining soft and hard targets.
    - Soft Target Loss: Kullback-Leibler divergence between the softened outputs of
      the student and teacher models.
    - Hard Target Loss: Standard cross-entropy loss with the ground truth labels.
    """
    soft_loss = nn.KLDivLoss(reduction='batchmean')(
        F.log_softmax(student_outputs / T, dim=1),
        F.softmax(teacher_outputs / T, dim=1)
    ) * (T * T)
    hard_loss = F.cross_entropy(student_outputs, labels)
    return alpha * hard_loss + (1.0 - alpha) * soft_loss

def calculate_loss(model, loader, loss_fn, is_distillation, teacher_model=None):
    """Calculates loss over a dataset without performing training steps."""
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for data, target in loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            if is_distillation and teacher_model:
                teacher_output = teacher_model(data)
                loss = loss_fn(output, target, teacher_output, TEMPERATURE, ALPHA)
            else:
                loss = loss_fn(output, target)
            total_loss += loss.item()
    return total_loss / len(loader)

def evaluate(model, loader):
    """Calculates the accuracy of a model on a given dataset."""
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    return 100 * correct / total

def train_and_validate(model, train_loader, val_loader, optimizer, loss_fn, epochs, patience, is_distillation, teacher_model=None):
    """
    Executes the complete training and validation loop with early stopping.
    The model with the best validation loss is saved and restored at the end.
    """
    best_val_loss = float('inf')
    patience_counter = 0
    best_model_state = None

    for epoch in range(epochs):
        model.train()
        if is_distillation and teacher_model:
            teacher_model.eval()

        with tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} Training", leave=False) as pbar:
            for data, target in pbar:
                data, target = data.to(DEVICE), target.to(DEVICE)
                optimizer.zero_grad()
                output = model(data)

                if is_distillation:
                    with torch.no_grad():
                        teacher_output = teacher_model(data)
                    loss = loss_fn(output, target, teacher_output, TEMPERATURE, ALPHA)
                else:
                    loss = loss_fn(output, target)

                loss.backward()
                optimizer.step()
                pbar.set_postfix(loss=loss.item())

        val_loss = calculate_loss(model, val_loader, loss_fn, is_distillation, teacher_model)
        val_acc = evaluate(model, val_loader)
        print(f"Epoch {epoch+1}/{epochs} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            best_model_state = copy.deepcopy(model.state_dict())
            print("  -> Validation loss improved, saving model.")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"  -> Early stopping triggered after {patience} epochs with no improvement.")
                break

    if best_model_state:
        model.load_state_dict(best_model_state)
    return model

## Experiment Execution

Now we will conduct the experiment in the usual three stages:
1.  **Evaluate the Teacher**: Load a pre-trained ResNet-56 model and measure its baseline accuracy on the CIFAR-10 test set. This model comes from the `pytorch-cifar-models` repository on GitHub and serves as our high-performance but computationally expensive expert.
2.  **Train Student from Scratch**: Train our `StudentNet` using only the standard cross-entropy loss. This provides a baseline for comparison.
3.  **Train Distilled Student**: Train the `StudentNet` again, but this time using the distillation loss function, which incorporates knowledge from the teacher model.

To ensure a fair comparison, both student models will start from the exact same randomly initialised weights.

In [ ]:
print("--- 1. Evaluating Teacher Model ---")
print("Loading pre-trained teacher model (cifar10_resnet56)...")
teacher_model = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar10_resnet56", pretrained=True)
teacher_model.to(DEVICE)
teacher_model.eval()

teacher_accuracy = evaluate(teacher_model, test_loader)
print(f"Teacher (ResNet-56) Test Accuracy: {teacher_accuracy:.2f}%")

--- 1. Evaluating Teacher Model ---
Loading pre-trained teacher model (cifar10_resnet56)...


/usr/local/lib/python3.11/dist-packages/torch/hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/chenyaofo/pytorch-cifar-models/zipball/master" to /root/.cache/torch/hub/master.zip
Downloading: "https://github.com/chenyaofo/pytorch-cifar-models/releases/download/resnet/cifar10_resnet56-187c023a.pt" to /root/.cache/torch/hub/checkpoints/cifar10_resnet56-187c023a.pt
100%|██████████| 3.39M/3.39M [00:00<00:00, 49.2MB/s

Teacher (ResNet-56) Test Accuracy: 94.37%


In [ ]:
# Create a template student and save its initial weights for a fair comparison
print("\nCreating a template student and saving its initial weights...")
template_student = StudentNet()
initial_student_state = copy.deepcopy(template_student.state_dict())

print("\n--- 2. Training Student from Scratch (Baseline) ---")
student_baseline = StudentNet().to(DEVICE)
student_baseline.load_state_dict(initial_student_state)
optimizer_baseline = optim.Adam(student_baseline.parameters(), lr=LR)
loss_fn_baseline = nn.CrossEntropyLoss()

student_baseline = train_and_validate(
    student_baseline, train_loader, val_loader, optimizer_baseline,
    loss_fn_baseline, EPOCHS, PATIENCE, is_distillation=False
)
baseline_accuracy = evaluate(student_baseline, test_loader)
print(f"\nFinal Baseline Student Test Accuracy: {baseline_accuracy:.2f}%")


Creating a template student and saving its initial weights...

--- 2. Training Student from Scratch (Baseline) ---


Epoch 1/20 Training:   0%|          | 0/704 [00:00<?, ?it/s]

Epoch 1/20 | Val Loss: 1.1110 | Val Acc: 60.62%
  -> Validation loss improved, saving model.


Epoch 2/20 Training:   0%|          | 0/704 [00:00<?, ?it/s]

Epoch 2/20 | Val Loss: 1.0160 | Val Acc: 62.54%
  -> Validation loss improved, saving model.


...

Epoch 8/20 | Val Loss: 1.0771 | Val Acc: 70.18%


Epoch 9/20 Training:   0%|          | 0/704 [00:00<?, ?it/s]

Epoch 9/20 | Val Loss: 1.1865 | Val Acc: 69.58%


Epoch 10/20 Training:   0%|          | 0/704 [00:00<?, ?it/s]

Epoch 10/20 | Val Loss: 1.3063 | Val Acc: 68.40%
  -> Early stopping triggered after 5 epochs with no improvement.

Final Baseline Student Test Accuracy: 70.86%


In [ ]:
print("\n--- 3. Training Distilled Student ---")
student_distilled = StudentNet().to(DEVICE)
student_distilled.load_state_dict(initial_student_state)
optimizer_distilled = optim.Adam(student_distilled.parameters(), lr=LR)
loss_fn_distilled = distillation_loss

student_distilled = train_and_validate(
    student_distilled, train_loader, val_loader, optimizer_distilled,
    loss_fn_distilled, EPOCHS, PATIENCE, is_distillation=True, teacher_model=teacher_model
)
distilled_accuracy = evaluate(student_distilled, test_loader)
print(f"\nFinal Distilled Student Test Accuracy: {distilled_accuracy:.2f}%")


--- 3. Training Distilled Student ---


Epoch 1/20 Training:   0%|          | 0/704 [00:00<?, ?it/s]

Epoch 1/20 | Val Loss: 5.7735 | Val Acc: 60.08%
  -> Validation loss improved, saving model.


Epoch 2/20 Training:   0%|          | 0/704 [00:00<?, ?it/s]

Epoch 2/20 | Val Loss: 5.1554 | Val Acc: 64.90%
  -> Validation loss improved, saving model.


...

Epoch 9/20 | Val Loss: 4.2734 | Val Acc: 73.30%


Epoch 10/20 Training:   0%|          | 0/704 [00:00<?, ?it/s]

Epoch 10/20 | Val Loss: 4.2872 | Val Acc: 73.34%


Epoch 11/20 Training:   0%|          | 0/704 [00:00<?, ?it/s]

Epoch 11/20 | Val Loss: 4.3367 | Val Acc: 73.08%
  -> Early stopping triggered after 5 epochs with no improvement.

Final Distilled Student Test Accuracy: 71.68%


## Final Results

In [ ]:
print("\n\n--- FINAL COMPREHENSIVE RESULTS ---")
print(f"Teacher (ResNet-56) Accuracy: {teacher_accuracy:.2f}%")
print(f"Student (Baseline) Accuracy:  {baseline_accuracy:.2f}%")
print(f"Student (Distilled) Accuracy: {distilled_accuracy:.2f}%")
print("---------------------------------")
improvement = distilled_accuracy - baseline_accuracy
print(f"Improvement with Distillation: {improvement:.2f}%")



--- FINAL COMPREHENSIVE RESULTS ---
Teacher (ResNet-56) Accuracy: 94.37%
Student (Baseline) Accuracy:  70.86%
Student (Distilled) Accuracy: 71.68%
---------------------------------
Improvement with Distillation: 0.82%


Running this many times and performing a statistical validation, we got the following results:

**Statistical Test (Paired t-test):**
*   **Improvement with Distillation:** 2.24%
*   **T-statistic:** 3.9024
*   **P-value:** 0.0175

**Conclusion:**
The improvement from knowledge distillation is **STATISTICALLY SIGNIFICANT**.

**Let's see whether a bigger task (e.g. with more classes) can benefit from distillation even more.**

# A More Challenging Task: CIFAR-100

We now transition to the CIFAR-100 dataset. This task is significantly more difficult as it contains 100 classes instead of 10. We will first attempt to distil knowledge into the same simple student architecture to observe the impact of distillation on a harder problem.

A key optimisation will be introduced here: pre-computing the teacher's outputs (logits). The new teacher model is a large ResNet-50 that requires upscaled input images. To avoid the computational expense of running this large model during every single training step of the student, we will run it once over the entire dataset and save its predictions. This creates a "lookup table" of logits, saving a significant amount of time during the student's training loop.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split, Dataset
from torchvision import datasets, transforms
from tqdm.notebook import tqdm
import numpy as np
import copy
import os
from transformers import AutoModelForImageClassification

# --- Configuration ---
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
EPOCHS = 30
PATIENCE = 5
BATCH_SIZE = 64
LR = 0.001
TEMPERATURE = 10
ALPHA = 0.2
SEED = 80
LOGITS_LOOKUP_PATH = 'teacher_logits_lookup.pt'

print(f"Using device: {DEVICE}")
print(f"Using random seed: {SEED}")

# --- Data Loading and Preprocessing for CIFAR-100 ---
student_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))
])

full_train_dataset = datasets.CIFAR100(root='./data', train=True, download=True, transform=student_transform)
test_dataset = datasets.CIFAR100(root='./data', train=False, download=True, transform=student_transform)

train_size = int(0.9 * len(full_train_dataset))
val_size = len(full_train_dataset) - train_size
generator = torch.Generator().manual_seed(SEED)
train_subset, val_subset = random_split(full_train_dataset, [train_size, val_size], generator=generator)

test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
print(f"Using CIFAR-100 dataset. Train: {len(train_subset)}, Val: {len(val_subset)}")

Using device: cuda
Using random seed: 80


100%|██████████| 169M/169M [00:03<00:00, 48.4MB/s]


Using CIFAR-100 dataset. Train: 45000, Val: 5000


## Model Architecture (Simple Student for CIFAR-100)
This is the same simple CNN architecture used for the CIFAR-10 task, but with its final layer adjusted to output 100 classes instead of 10.

In [ ]:
class StudentNet(nn.Module):
    def __init__(self):
        super(StudentNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, 100) # 100 instead of 10, as we are using CIFAR-100

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

## Pre-computation and Distillation Setup
Here we define the helper functions and classes needed for our pre-computation strategy.

* `get_teacher_logits_lookup`: A function that iterates through the entire dataset once, using the powerful teacher model to generate predictions (logits) for every single image. It returns a tensor that acts as a lookup table.
* `DistillationDataset`: A custom PyTorch Dataset class. It wraps our original data subset and, for each image, also retrieves the corresponding pre-computed teacher logit from the lookup table. This allows the DataLoader to provide the student with the image, the true label, and the teacher's guidance all at once.

In [ ]:
@torch.no_grad()
def get_teacher_logits_lookup(model, full_dataset, device):
    """Computes all logits and maps them to their original dataset index."""
    lookup_table = torch.zeros(len(full_dataset), 100) # 100 classes for CIFAR-100
    loader = DataLoader(full_dataset, batch_size=BATCH_SIZE, shuffle=False)
    model.eval()

    current_pos = 0
    for data, _ in tqdm(loader, desc="Computing Logit Lookup Table"):
        data = data.to(device)
        # Teacher model expects 224x224 images, so we resize them
        data = F.interpolate(data, size=(224, 224), mode='bilinear', align_corners=False)
        logits = model(data).logits.cpu()

        batch_size = data.size(0)
        lookup_table[current_pos:current_pos + batch_size] = logits
        current_pos += batch_size

    return lookup_table

class DistillationDataset(Dataset):
    """A custom dataset to pair images with pre-computed teacher logits."""
    def __init__(self, original_subset, teacher_logits_lookup):
        self.original_subset = original_subset
        self.teacher_logits_lookup = teacher_logits_lookup

    def __len__(self):
        return len(self.original_subset)

    def __getitem__(self, idx):
        image, label = self.original_subset[idx]
        # Use the original index to find the correct pre-computed logit
        original_idx = self.original_subset.indices[idx]
        teacher_logit = self.teacher_logits_lookup[original_idx]
        return image, label, teacher_logit

## Training and Evaluation Functions
These functions are adapted from the CIFAR-10 experiment. The key change is that they can now handle the different data formats from a standard `DataLoader` (image, label) and our new `DistillationDataset` `DataLoader` (image, label, pre-computed_logit).

In [ ]:
def distillation_loss(student_outputs, labels, precomputed_teacher_logits, T, alpha):
    soft_loss = nn.KLDivLoss(reduction='batchmean')(
        F.log_softmax(student_outputs / T, dim=1),
        F.softmax(precomputed_teacher_logits / T, dim=1)
    ) * (T * T)
    hard_loss = F.cross_entropy(student_outputs, labels)
    return alpha * hard_loss + (1.0 - alpha) * soft_loss

def calculate_loss(model, loader, loss_fn, is_distillation):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in loader:
            if is_distillation:
                data, target, teacher_logits = batch
                teacher_logits = teacher_logits.to(DEVICE)
            else:
                data, target = batch
            data, target = data.to(DEVICE), target.to(DEVICE)

            output = model(data)

            if is_distillation:
                loss = loss_fn(output, target, teacher_logits, TEMPERATURE, ALPHA)
            else:
                loss = loss_fn(output, target)
            total_loss += loss.item()
    return total_loss / len(loader)

def train_and_validate(model, train_loader, val_loader, optimizer, loss_fn, epochs, patience, is_distillation):
    best_val_loss = float('inf')
    patience_counter = 0
    best_model_state = None

    for epoch in range(epochs):
        model.train()
        with tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} Training", leave=False) as pbar:
            for batch in pbar:
                if is_distillation:
                    data, target, teacher_logits = batch
                    teacher_logits = teacher_logits.to(DEVICE)
                else:
                    data, target = batch
                data, target = data.to(DEVICE), target.to(DEVICE)

                optimizer.zero_grad()
                output = model(data)

                if is_distillation:
                    loss = loss_fn(output, target, teacher_logits, TEMPERATURE, ALPHA)
                else:
                    loss = loss_fn(output, target)

                loss.backward()
                optimizer.step()
                pbar.set_postfix(loss=loss.item())

        val_loss = calculate_loss(model, val_loader, loss_fn, is_distillation)
        val_acc = evaluate(model, val_loader)
        print(f"Epoch {epoch+1}/{epochs} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            best_model_state = copy.deepcopy(model.state_dict())
            print("  -> Validation loss improved, saving model.")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"  -> Early stopping triggered after {patience} epochs with no improvement.")
                break

    if best_model_state:
        model.load_state_dict(best_model_state)
    return model

def evaluate(model, loader, is_teacher=False):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in loader:
            # The data format is different for our distillation loader
            data, target = batch[0], batch[1]
            data, target = data.to(DEVICE), target.to(DEVICE)

            if is_teacher:
                data = F.interpolate(data, size=(224, 224), mode='bilinear', align_corners=False)
                outputs = model(data).logits
            else:
                outputs = model(data)

            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    return 100 * correct / total

## Experiment 1: Distillation on CIFAR-100 with a Simple Student
We now execute the full experiment pipeline:
* Evaluate Teacher: Load a pre-trained ResNet-50 model from Hugging Face (sourced directly from the Hugging Face Model Hub – `jialicheng/cifar100-resnet-50`) and measure its accuracy on the CIFAR-100 test set.
* Pre-compute Logits: If not already saved, compute the teacher's logit for every image in the training set and save them to a file (`teacher_logits_lookup.pt`).
* Train Baseline Student: Train the simple StudentNet from scratch using standard cross-entropy loss.
* Train Distilled Student: Train an identical StudentNet using the distillation loss, guided by the pre-computed teacher logits.

To ensure a fair comparison, both student models start from the exact same randomly initialised weights. After training, the teacher model is deleted to free up GPU memory.

In [ ]:
# --- Main Experiment ---
teacher_model = AutoModelForImageClassification.from_pretrained("jialicheng/cifar100-resnet-50")
teacher_model.to(DEVICE)
teacher_model.eval()

print("\n--- Evaluating Teacher Model ---")
teacher_accuracy = evaluate(teacher_model, test_loader, is_teacher=True)
print(f"Teacher (ResNet-50) Test Accuracy: {teacher_accuracy:.2f}%")

if os.path.exists(LOGITS_LOOKUP_PATH):
    print(f"\n--- Loading pre-computed logit lookup table from {LOGITS_LOOKUP_PATH} ---")
    teacher_logits_lookup = torch.load(LOGITS_LOOKUP_PATH)
else:
    print("\n--- Pre-computing teacher logit lookup table (this may take a while)... ---")
    teacher_logits_lookup = get_teacher_logits_lookup(teacher_model, full_train_dataset, DEVICE)
    print(f"--- Saving computed lookup table to {LOGITS_LOOKUP_PATH} for future use... ---")
    torch.save(teacher_logits_lookup, LOGITS_LOOKUP_PATH)

print(f"Loaded {len(teacher_logits_lookup)} total logits into lookup table.")

distill_train_dataset = DistillationDataset(train_subset, teacher_logits_lookup)
distill_val_dataset = DistillationDataset(val_subset, teacher_logits_lookup)
distill_train_loader = DataLoader(distill_train_dataset, batch_size=BATCH_SIZE, shuffle=True)
distill_val_loader = DataLoader(distill_val_dataset, batch_size=BATCH_SIZE, shuffle=False)

del teacher_model
torch.cuda.empty_cache()

print("\n--- Training Student from Scratch (Baseline) ---")
template_student = StudentNet()
initial_student_state = copy.deepcopy(template_student.state_dict())

student_baseline = StudentNet().to(DEVICE)
student_baseline.load_state_dict(initial_student_state)
optimizer_baseline = optim.Adam(student_baseline.parameters(), lr=LR)
loss_fn_baseline = nn.CrossEntropyLoss()

baseline_train_loader = DataLoader(train_subset, batch_size=BATCH_SIZE, shuffle=True)
baseline_val_loader = DataLoader(val_subset, batch_size=BATCH_SIZE, shuffle=False)

student_baseline = train_and_validate(
    student_baseline, baseline_train_loader, baseline_val_loader, optimizer_baseline,
    loss_fn_baseline, EPOCHS, PATIENCE, is_distillation=False
)
baseline_accuracy = evaluate(student_baseline, test_loader)
print(f"Final Baseline Student Test Accuracy: {baseline_accuracy:.2f}%")

print("\n--- Training Distilled Student (with pre-computed logits) ---")
student_distilled = StudentNet().to(DEVICE)
student_distilled.load_state_dict(initial_student_state)
optimizer_distilled = optim.Adam(student_distilled.parameters(), lr=LR)
loss_fn_distilled = distillation_loss

student_distilled = train_and_validate(
    student_distilled, distill_train_loader, distill_val_loader, optimizer_distilled,
    loss_fn_distilled, EPOCHS, PATIENCE, is_distillation=True
)
distilled_accuracy = evaluate(student_distilled, test_loader)
print(f"Final Distilled Student Test Accuracy: {distilled_accuracy:.2f}%")

print("\n\n--- FINAL COMPREHENSIVE RESULTS (Simple Student) ---")
print(f"Teacher (ResNet-50) Accuracy: {teacher_accuracy:.2f}%")
print(f"Student (Baseline) Accuracy:  {baseline_accuracy:.2f}%")
print(f"Student (Distilled) Accuracy: {distilled_accuracy:.2f}%")
print("---------------------------------")
improvement = distilled_accuracy - baseline_accuracy
print(f"Improvement with Distillation: {improvement:.2f}%")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/95.1M [00:00<?, ?B/s]


--- Evaluating Teacher Model ---
Teacher (ResNet-50) Test Accuracy: 82.75%

--- Pre-computing teacher logit lookup table (this may take a while)... ---


Computing Logit Lookup Table:   0%|          | 0/782 [00:00<?, ?it/s]

...

Epoch 10/30 | Val Loss: 2.2723 | Val Acc: 42.64%


Epoch 11/30 Training:   0%|          | 0/704 [00:00<?, ?it/s]

Epoch 11/30 | Val Loss: 2.2735 | Val Acc: 43.12%


Epoch 12/30 Training:   0%|          | 0/704 [00:00<?, ?it/s]

Epoch 12/30 | Val Loss: 2.3236 | Val Acc: 42.46%
  -> Early stopping triggered after 5 epochs with no improvement.
Final Distilled Student Test Accuracy: 45.46%


--- FINAL COMPREHENSIVE RESULTS (Simple Student) ---
Teacher (ResNet-50) Accuracy: 82.75%
Student (Baseline) Accuracy:  39.86%
Student (Distilled) Accuracy: 45.46%
---------------------------------
Improvement with Distillation: 5.60%


## Experiment 2: A More Capable Student (ResNet-18)

The results from our simple student are promising and confirm that the distillation process is effective. However, the student's performance was ultimately capped by the inherent limitations of its simple CNN architecture. It simply lacked the capacity to fully absorb the rich guidance provided by the powerful ResNet-50 teacher.

To bridge this gap, our next experiment will use a more capable **ResNet-18** as the student model. To unlock its full potential, we will also employ more robust training techniques.

### Aligning Data Augmentation with Pre-computed Logits

Data augmentation (like random crops and horizontal flips) is crucial for helping a model generalise. However, it creates a conflict with our pre-computation strategy. The teacher's logits were calculated on the *original* images, while data augmentation creates a slightly different version of each image for every training epoch.

We solve this with a more sophisticated data pipeline and training loop:

*   **On-the-Fly Transformation with PIL:** Instead of pre-transforming our dataset, we load the images as their raw **PIL** (Python Imaging Library, the standard for image handling in Python) objects. Our custom data loaders then apply the random augmentations to each PIL image "on-the-fly" during the training step.

*   **Correct Logit Mapping:** Our redesigned `DistillationDataset` ensures a perfect match. For each image, it applies the random augmentation to create the student's input, but uses the image's original, unchanged index to retrieve the corresponding logit from our pre-computed lookup table. This guarantees the student is always guided by the correct teacher signal.

In the following, we'll also use a **Learning Rate Scheduling** (Cosine Annealing Scheduler) to dynamically adjust the learning rate during training. This method starts with a higher learning rate and smoothly decreases it over the epochs, which often helps the model to settle into a better and more stable final solution.

This enhanced training process is significantly more computationally intensive and will take a considerable amount of time to complete.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split, Dataset
from torchvision import datasets, transforms
from PIL import Image
from tqdm.notebook import tqdm
import numpy as np
import copy
import os
from transformers import AutoModelForImageClassification

# --- Configuration ---
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
EPOCHS = 1000
PATIENCE = 10
BATCH_SIZE = 64
LR = 0.001
TEMPERATURE = 10
ALPHA = 0.2
SEED = 45
LOGITS_LOOKUP_PATH = 'teacher_logits_lookup.pt'

print(f"Using device: {DEVICE}")
print(f"Using random seed for data split: {SEED}")

# --- Data Loading and Preprocessing for CIFAR-100 ---
# Student transform includes data augmentation for training
student_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))
])

# Evaluation transform does not use augmentation
eval_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))
])

# Load datasets WITHOUT transforms to get PIL Images
full_train_dataset_pil = datasets.CIFAR100(root='./data', train=True, download=True, transform=None)
test_dataset_pil = datasets.CIFAR100(root='./data', train=False, download=True, transform=None)

# Create splits from the PIL-based dataset
train_size = int(0.9 * len(full_train_dataset_pil))
val_size = len(full_train_dataset_pil) - train_size
generator = torch.Generator().manual_seed(SEED)
train_subset_pil, val_subset_pil = random_split(full_train_dataset_pil, [train_size, val_size], generator=generator)

print(f"Using CIFAR-100 dataset. Train: {len(train_subset_pil)}, Val: {len(val_subset_pil)}")

Using device: cuda
Using random seed for data split: 45
Using CIFAR-100 dataset. Train: 45000, Val: 5000


## Model Architecture (ResNet-18 Student)
We define the building blocks and the final architecture for a ResNet-18 model, adapted for the 32x32 images and 100 classes of the CIFAR-100 dataset.

In [ ]:
class BasicBlock(nn.Module):
    """Building Block for ResNet-18/34."""
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class StudentNet(nn.Module):
    """ResNet-18 model adapted for CIFAR-100."""
    def __init__(self, block=BasicBlock, num_blocks=[2, 2, 2, 2], num_classes=100):
        super(StudentNet, self).__init__()
        self.in_planes = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride_val in strides:
            layers.append(block(self.in_planes, planes, stride_val))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

## Advanced Data Handling and Training
The data handling and training functions are updated to accommodate data augmentation and the learning rate scheduler.

* `TransformedDataset`: A new wrapper class that applies a given transformation to a PIL-based dataset on-the-fly. This is crucial for applying data augmentation to training data while using non-augmented data for validation and testing.
* `DistillationDataset`: This class is updated to accept a transform, ensuring that even when we retrieve a pre-computed logit, we can apply data augmentation to the corresponding image.
* `train_and_validate`: The main training loop now incorporates a `CosineAnnealingLR` scheduler, which is stepped at the end of each epoch. The current learning rate is printed in the logs.


In [ ]:
class TransformedDataset(Dataset):
    """A wrapper to apply transformations to a subset of PIL images."""
    def __init__(self, subset, transform):
        self.subset = subset
        self.transform = transform

    def __getitem__(self, index):
        x, y = self.subset[index]
        if self.transform:
            x = self.transform(x)
        return x, y

    def __len__(self):
        return len(self.subset)

@torch.no_grad()
def get_teacher_logits_lookup(model, full_pil_dataset, transform, device):
    """Computes all logits from the original, untransformed dataset."""
    lookup_table = torch.zeros(len(full_pil_dataset), 100)
    transformed_dataset = TransformedDataset(full_pil_dataset, transform)
    loader = DataLoader(transformed_dataset, batch_size=BATCH_SIZE, shuffle=False)
    model.eval()

    current_pos = 0
    for data, _ in tqdm(loader, desc="Computing Logit Lookup Table"):
        data = data.to(device)
        data = F.interpolate(data, size=(224, 224), mode='bilinear', align_corners=False)
        logits = model(data).logits.cpu()
        batch_size = data.size(0)
        lookup_table[current_pos:current_pos + batch_size] = logits
        current_pos += batch_size
    return lookup_table

class DistillationDataset(Dataset):
    """Pairs PIL images with logits and applies transforms on-the-fly."""
    def __init__(self, original_subset_pil, teacher_logits_lookup, transform=None):
        self.original_subset_pil = original_subset_pil
        self.teacher_logits_lookup = teacher_logits_lookup
        self.transform = transform

    def __len__(self):
        return len(self.original_subset_pil)

    def __getitem__(self, idx):
        image, label = self.original_subset_pil[idx]
        if self.transform:
            image = self.transform(image)
        original_idx = self.original_subset_pil.indices[idx]
        teacher_logit = self.teacher_logits_lookup[original_idx]
        return image, label, teacher_logit

def distillation_loss(student_outputs, labels, precomputed_teacher_logits, T, alpha):
    soft_loss = nn.KLDivLoss(reduction='batchmean')(
        F.log_softmax(student_outputs / T, dim=1),
        F.softmax(precomputed_teacher_logits / T, dim=1)
    ) * (T * T)
    hard_loss = F.cross_entropy(student_outputs, labels)
    return alpha * hard_loss + (1.0 - alpha) * soft_loss

def calculate_loss(model, loader, loss_fn, is_distillation):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in loader:
            if is_distillation:
                data, target, teacher_logits = batch
                teacher_logits = teacher_logits.to(DEVICE)
            else:
                data, target = batch
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            if is_distillation:
                loss = loss_fn(output, target, teacher_logits, TEMPERATURE, ALPHA)
            else:
                loss = loss_fn(output, target)
            total_loss += loss.item()
    return total_loss / len(loader)

def train_and_validate(model, train_loader, val_loader, optimizer, loss_fn, epochs, patience, is_distillation):
    best_val_loss = float('inf')
    patience_counter = 0
    best_model_state = None
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)

    for epoch in range(epochs):
        model.train()
        with tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} Training", leave=False) as pbar:
            for batch in pbar:
                if is_distillation:
                    data, target, teacher_logits = batch
                    teacher_logits = teacher_logits.to(DEVICE)
                else:
                    data, target = batch
                data, target = data.to(DEVICE), target.to(DEVICE)

                optimizer.zero_grad()
                output = model(data)

                if is_distillation:
                    loss = loss_fn(output, target, teacher_logits, TEMPERATURE, ALPHA)
                else:
                    loss = loss_fn(output, target)

                loss.backward()
                optimizer.step()
                pbar.set_postfix(loss=loss.item())

        val_loss = calculate_loss(model, val_loader, loss_fn, is_distillation)
        val_acc = evaluate(model, val_loader)
        print(f"Epoch {epoch+1}/{epochs} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}% | LR: {scheduler.get_last_lr()[0]:.6f}")

        scheduler.step()

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            best_model_state = copy.deepcopy(model.state_dict())
            print("  -> Validation loss improved, saving model.")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"  -> Early stopping triggered after {patience} epochs with no improvement.")
                break

    if best_model_state:
        model.load_state_dict(best_model_state)
    return model

def evaluate(model, loader, is_teacher=False):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in loader:
            data, target = batch[0], batch[1]
            data, target = data.to(DEVICE), target.to(DEVICE)

            if is_teacher:
                data = F.interpolate(data, size=(224, 224), mode='bilinear', align_corners=False)
                outputs = model(data).logits
            else:
                outputs = model(data)

            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    return 100 * correct / total

## Experiment 2: Execution with ResNet-18 Student
We now run the final, most intensive experiment.

A crucial step for a fair comparison is added: we create a single "template" ResNet-18 model and save its initial, randomly-generated weights. Both the baseline and the distilled student models will be loaded with these exact same initial weights before their respective training begins. 

The pipeline is as follows:

1. Evaluate the teacher model's performance.
2. Load the pre-computed teacher logits (if the file exists) or compute them. The computation now uses the `eval_transform` to ensure logits are calculated on clean, non-augmented images.
3. Create and save a common initial state for the ResNet-18 student models.
4. Train the baseline ResNet-18 student using data augmentation.
5. Train the distilled ResNet-18 student, also using data augmentation, but with the distillation loss function.
6. Compare the final test accuracies.

In [ ]:
# --- Main Experiment ---
teacher_model = AutoModelForImageClassification.from_pretrained("jialicheng/cifar100-resnet-50")
teacher_model.to(DEVICE)
teacher_model.eval()

print("\n--- Evaluating Teacher Model ---")
teacher_test_loader = DataLoader(TransformedDataset(test_dataset_pil, eval_transform), batch_size=BATCH_SIZE)
teacher_accuracy = evaluate(teacher_model, teacher_test_loader, is_teacher=True)
print(f"Teacher (ResNet-50) Test Accuracy: {teacher_accuracy:.2f}%")

if os.path.exists(LOGITS_LOOKUP_PATH):
    print(f"\n--- Loading pre-computed logit lookup table from {LOGITS_LOOKUP_PATH} ---")
    teacher_logits_lookup = torch.load(LOGITS_LOOKUP_PATH)
else:
    print("\n--- Pre-computing teacher logit lookup table (this will take a while)... ---")
    # Logits are computed on the clean, non-augmented data
    teacher_logits_lookup = get_teacher_logits_lookup(teacher_model, full_train_dataset_pil, eval_transform, DEVICE)
    print(f"--- Saving computed lookup table to {LOGITS_LOOKUP_PATH} for future use... ---")
    torch.save(teacher_logits_lookup, LOGITS_LOOKUP_PATH)

print(f"Loaded {len(teacher_logits_lookup)} total logits into lookup table.")

del teacher_model
torch.cuda.empty_cache()

print("\n--- Creating and saving a common initial state for student models ---")
initial_student_model = StudentNet().to(DEVICE)
initial_student_state_dict = copy.deepcopy(initial_student_model.state_dict())
print("Initial state saved. This will be used for both baseline and distilled students.")
del initial_student_model
torch.cuda.empty_cache()


print("\n--- Training Student from Scratch (Baseline) ---")
student_baseline = StudentNet().to(DEVICE)
student_baseline.load_state_dict(initial_student_state_dict)

optimizer_baseline = optim.Adam(student_baseline.parameters(), lr=LR)
loss_fn_baseline = nn.CrossEntropyLoss()

# Create baseline datasets by applying the correct transforms to the PIL subsets
baseline_train_dataset = TransformedDataset(train_subset_pil, student_transform)
baseline_val_dataset = TransformedDataset(val_subset_pil, eval_transform)
baseline_train_loader = DataLoader(baseline_train_dataset, batch_size=BATCH_SIZE, shuffle=True)
baseline_val_loader = DataLoader(baseline_val_dataset, batch_size=BATCH_SIZE, shuffle=False)

student_baseline = train_and_validate(
    student_baseline, baseline_train_loader, baseline_val_loader, optimizer_baseline,
    loss_fn_baseline, EPOCHS, PATIENCE, is_distillation=False
)
final_test_loader = DataLoader(TransformedDataset(test_dataset_pil, eval_transform), batch_size=BATCH_SIZE)
baseline_accuracy = evaluate(student_baseline, final_test_loader)
print(f"Final Baseline Student (ResNet-18) Test Accuracy: {baseline_accuracy:.2f}%")


print("\n--- Training Distilled Student (with pre-computed logits) ---")
student_distilled = StudentNet().to(DEVICE)
student_distilled.load_state_dict(initial_student_state_dict)

optimizer_distilled = optim.Adam(student_distilled.parameters(), lr=LR)
loss_fn_distilled = distillation_loss

# Create distillation datasets, applying the correct transforms
distill_train_dataset = DistillationDataset(train_subset_pil, teacher_logits_lookup, transform=student_transform)
distill_val_dataset = DistillationDataset(val_subset_pil, teacher_logits_lookup, transform=eval_transform)
distill_train_loader = DataLoader(distill_train_dataset, batch_size=BATCH_SIZE, shuffle=True)
distill_val_loader = DataLoader(distill_val_dataset, batch_size=BATCH_SIZE, shuffle=False)

student_distilled = train_and_validate(
    student_distilled, distill_train_loader, distill_val_loader, optimizer_distilled,
    loss_fn_distilled, EPOCHS, PATIENCE, is_distillation=True
)
distilled_accuracy = evaluate(student_distilled, final_test_loader)
print(f"Final Distilled Student (ResNet-18) Test Accuracy: {distilled_accuracy:.2f}%")


--- Evaluating Teacher Model ---
Teacher (ResNet-50) Test Accuracy: 82.75%

--- Loading pre-computed logit lookup table from teacher_logits_lookup.pt ---
Loaded 50000 total logits into lookup table.

--- Creating and saving a common initial state for student models ---
Initial state saved. This will be used for both baseline and distilled students.

--- Training Student from Scratch (Baseline) ---


Epoch 1/1000 Training:   0%|          | 0/704 [00:00<?, ?it/s]

Epoch 1/1000 | Val Loss: 3.5614 | Val Acc: 15.14% | LR: 0.001000
  -> Validation loss improved, saving model.


Epoch 2/1000 Training:   0%|          | 0/704 [00:00<?, ?it/s]

Epoch 2/1000 | Val Loss: 3.0496 | Val Acc: 25.78% | LR: 0.001000
  -> Validation loss improved, saving model.


...

Epoch 135/1000 | Val Loss: 0.6004 | Val Acc: 79.06% | LR: 0.000956


Epoch 136/1000 Training:   0%|          | 0/704 [00:00<?, ?it/s]

Epoch 136/1000 | Val Loss: 0.5990 | Val Acc: 78.98% | LR: 0.000956


Epoch 137/1000 Training:   0%|          | 0/704 [00:00<?, ?it/s]

Epoch 137/1000 | Val Loss: 0.6044 | Val Acc: 78.34% | LR: 0.000955
  -> Early stopping triggered after 10 epochs with no improvement.
Final Distilled Student (ResNet-18) Test Accuracy: 77.74%


## Final Results (ResNet-18 Student)
Finally, we compare the performance of the powerful ResNet-50 teacher, our baseline ResNet-18 student, and the distilled ResNet-18 student.

In [ ]:
print("\n\n--- FINAL COMPREHENSIVE RESULTS (ResNet-18 Student) ---")
print(f"Teacher (ResNet-50) Accuracy: {teacher_accuracy:.2f}%")
print(f"Student (Baseline - ResNet18) Accuracy:  {baseline_accuracy:.2f}%")
print(f"Student (Distilled - ResNet18) Accuracy: {distilled_accuracy:.2f}%")
print("---------------------------------")
improvement = distilled_accuracy - baseline_accuracy
print(f"Improvement with Distillation: {improvement:.2f}%")



--- FINAL COMPREHENSIVE RESULTS (ResNet-18 Student) ---
Teacher (ResNet-50) Accuracy: 82.75%
Student (Baseline - ResNet18) Accuracy:  63.94%
Student (Distilled - ResNet18) Accuracy: 77.74%
---------------------------------
Improvement with Distillation: 13.80%


## Key Observations

Some critical insights emerge from these experiments:

*  **Distillation is More Impactful on Complex Tasks:** The accuracy improvement from distillation was notably greater on the 100-class CIFAR-100 dataset than on the 10-class CIFAR-10. This suggests that the value of the teacher's "dark knowledge" is magnified when the classification task is more complex and the decision boundaries are harder to define.

*  **Bridging the Performance Gap:** The final experiment with the ResNet-18 student is particularly compelling. Despite being a significantly smaller and more computationally efficient model than the ResNet-50 teacher, the distilled student achieved a test accuracy that nearly closed the performance gap between them. This powerfully illustrates the primary promise of knowledge distillation: to create compact, efficient models that retain the high performance of much larger counterparts, making them viable for deployment in resource-constrained environments.

* **Distillation Enables More Stable and Prolonged Training**: The logs clearly show that the distilled student model continued to improve its validation loss for many more epochs compared to the baseline student, which quickly overfit and triggered early stopping. This demonstrates that the teacher's soft targets act as a powerful regulariser, providing a consistent learning signal that prevents the student from simply memorising the hard labels. The resulting accuracy improvement on the complex CIFAR-100 task is a direct result of this guided and more robust training process.

# Concluding Remarks and Future Directions

The experiments in this notebook successfully demonstrate the core principles and effectiveness of knowledge distillation. By transferring knowledge from a large, pre-trained teacher model to smaller student models using softened logits, we observed significant performance improvements across both the CIFAR-10 and CIFAR-100 datasets.

**However, this implementation explores only one form of distillation signal**. Further elements which could easily be added to the current notebook are the following (but there are many more!):

*   **Intermediate Feature Matching:** A highly effective technique involves compelling the student to mimic the teacher's internal "thought process". This can be achieved by adding a loss term that minimises the difference between the feature maps of the student's intermediate layers and those of the teacher. For instance, one could match the output of the final feature extraction block in the student's `StudentNet` with the corresponding block in the teacher, forcing the student to learn the teacher's method of feature representation, not just its final prediction.

*   **Dynamic Distillation Schedules:** The influence of the different loss components does not have to be static. Advanced methods often involve scheduling the hyperparameters during training. For example, the `alpha` parameter could be annealed over time, starting with a high reliance on the teacher's guidance (`alpha` is low) and gradually increasing the weight of the ground-truth labels as the student model becomes more capable.
  
We anticipate that these and other advanced techniques might not show a big improvement like the one we have seen here, but they are likely to further enhance the student's ability to generalise and learn from the teacher's knowledge, **maybe even reaching or surpassing the teacher's performance.**

Indeed, we have performed some preliminary tests with intermediate feature matching and got slightly better results, but we have not included them here to keep the focus on the core principles and because the improvements were not as significant as the ones we have here (and feature matching is also a lot more computationally expensive, especially in its refined forms).

One fascinating aspect we can think of is distillation for LLMs: intuitively, as we’ve moved to CIFAR-100, the task has become more complex and the teacher's knowledge more valuable. This suggests that distillation could be even more powerful in the context of LLMs, where next-token prediction is essentially a very large classification task over the vocabulary. Indeed, KD for LLMs exactly started by levaeraging the distillation signal originally proposed by Hinton – the one we have used here.

> Knowledge distillation remains something of an art: you need to experiment and find the right balance of models, data, and techniques for your specific task to achieve optimal results.